In [5]:
#Inputs: List of initial models to use, desired image folder (H&E vs UnremovedTranscripts), drg4 image path, manual drg4 path
#Outputs: Metrics csvs corresponding to each trained model
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import re
import cv2
from PIL import Image
from glob import glob
from read_roi import read_roi_file, read_roi_zip
from cellpose import core, utils, io, models, metrics, plot, train
import time
import random

# Set seed for reproducibility
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

# Optional: for PyTorch-based models (Cellpose may use this internally)
import torch
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

%run /media/yhs/5596744f-db7c-442f-9235-d0c9d50c0a6b/Cellpose/Finalized\ Code/Training/evaluationPipeline.ipynb

start_time = time.time()

use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')

#Original Directory
base_dir = '/media/yhs/5596744f-db7c-442f-9235-d0c9d50c0a6b/Cellpose/Batch2/DRG123'

#initial_model = 'CPx'
initial_model = 'cyto2'

#'H&EStain' or 'UnremovedTranscripts'
img_folder = 'UnremovedTranscripts'
txt_folder = 'UnremovedTranscriptsTxt'
min_masks = 5
c1 = "Red" #@param ["Grayscale", "Blue", "Green", "Red"]
c2_list = ["None", "Green"] #@param ["None", "Blue", "Green", "Red"]

#@markdown threshold on flow error to accept a mask (set higher to get more cells, e.g. in range from (0.1, 3.0), OR set to 0.0 to turn off so no cells discarded):
flow_threshold = 0.4 #@param {type:"slider", min:0.0, max:3.0, step:0.1}
#@markdown threshold on cellprob output to seed cell masks (set lower to include more pixels or higher to include fewer, e.g. in range from (-6, 6)):
cellprob_threshold = 0 #@param {type:"slider", min:-6, max:6, step:1}

def extracting_rois(file_path, img_path):
    with open(file_path, 'r') as file:
        lines = file.read().splitlines()
    rois = []
    img = Image.open(img_path)
    width, height = img.size
    for i in lines:
        rois.append(list(map(float, i.split())))
    rois_filtered = []
    for i in rois:
        rois_filtered.append(i[1:])
    int_rois = []
    for i in rois_filtered:
        count = 0
        x = []
        y = []
        for j in i:
            if count%2:
                x.append(int(round(j*width)))
            else:
                y.append(int(round(j*height)))
            count += 1
        temp = list(zip(x, y))
        filt = []
        for i in temp:
            if i not in filt:
                filt.append(i)
        int_rois.append(filt)
    return int_rois

def save_rois_with_conversion(masks, files, output_dir):
    # Ensure all masks are in the correct format
    masks = [mask.astype(np.uint16) if mask.dtype != np.uint16 else mask for mask in masks]
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    count = 0
    for file, mask in zip(files, masks):
        count += 1
        output_path = os.path.join(output_dir, file.split('/')[-1].replace('.jpg',''))
        io.save_rois(mask, output_path)

def zip_to_csv(directory):
    for filename in [i for i in os.listdir(directory) if '.zip' in i]:
        df_data = {'Name': [], 'X': [], 'Y': []}
        file_path = os.path.join(directory, filename)
        data = read_roi_zip(file_path)
        for name, roi in data.items():
            df_data['X'].extend(roi['x'])
            df_data['Y'].extend(roi['y'])
            for i in range(len(roi['x'])):
                df_data['Name'].append(str(roi['name']))
        df = pd.DataFrame.from_dict(df_data)
        out_dir = os.path.join(directory, 'csv')
        os.makedirs(out_dir, exist_ok=True)
        df.to_csv(os.path.join(out_dir, filename).replace('.zip','.csv').replace('_rois','').replace('Nonseg','Seg'), sep=',', index=False, header=True)

def csv_png(filename, height, width):
    rois = pd.read_csv(filename)
    image_height = height
    image_width = width
    grouped_rois = rois.groupby('Name')
    count = 1
    masks = []
    for id, group in grouped_rois:
        single_mask = np.zeros((image_height, image_width), dtype=np.uint8)
        polygon = [(x, y) for x, y in zip(group['X'], group['Y'])]
        pts = np.array(polygon, dtype=np.int32)
        pts = pts.reshape((-1, 1, 2))
        cv2.fillPoly(single_mask, [pts], color=count)
        masks.append(single_mask)
        count += 1
    if len(masks) == 0:
        return np.zeros((width, height), dtype=np.uint16)
    masks_array = np.stack(masks, axis=0)
    combined = np.max(masks_array, axis = 0)
    return combined

def folder_mapping(i_dir, j_dir):
    aligned = []
    for i in os.listdir(i_dir):
        for j in os.listdir(j_dir):
            if i.split()[0] == j.split()[0]:
                aligned.append((i, j))
    return aligned

def list_mapping(i_dir, j_dir):
    aligned = []
    for i in i_dir:
        for j in j_dir:
            if i.split('.')[0] == j.split('.')[0]:
                aligned.append((i, j))
            elif re.findall(r'\d+', i) == re.findall(r'\d+', j):
                aligned.append((i, j))
    print(aligned)
    return aligned

def check_unused_img(un_dir, u_dir, min_masks):
    for maskname in [i for i in os.listdir(un_dir) if '_masks' in i]:
        m_path = os.path.join(un_dir, maskname)
        marr = np.array(Image.open(m_path))
        num_masks = np.max(marr)
        if num_masks >= min_masks:
            os.rename(m_path, os.path.join(u_dir, maskname))
            os.rename(m_path.replace('_masks.png', '.jpg'), os.path.join(u_dir, maskname.replace('_masks.png', '.jpg')))

def check_unused_csv(un_dir, u_dir, min_masks):
    for csvname in [i for i in os.listdir(un_dir) if '.csv' in i]:
        c_path = os.path.join(un_dir, csvname)
        csv = pd.read_csv(c_path)
        if len(csv.groupby('Name')) >= min_masks:
            os.rename(c_path, os.path.join(u_dir, csvname))
            os.rename(c_path.replace('.csv', '.txt'), os.path.join(u_dir, csvname.replace('.csv', '.txt')))

def add_unused_csv(un_dir, u_dir, min_masks):
    for csvname in [i for i in os.listdir(u_dir) if '.csv' in i]:
        c_path = os.path.join(u_dir, csvname)
        csv = pd.read_csv(c_path)
        if len(csv.groupby('Name')) < min_masks:
            os.rename(c_path, os.path.join(un_dir, csvname))
            os.rename(c_path.replace('.csv', '.txt'), os.path.join(un_dir, csvname.replace('.csv', '.txt')))

def add_unused_img(un_dir, u_dir, min_masks):
    for maskname in [i for i in os.listdir(u_dir) if '_masks' in i]:
        m_path = os.path.join(u_dir, maskname)
        marr = np.array(Image.open(m_path))
        num_masks = np.max(marr)
        if num_masks < min_masks:
            os.rename(m_path, os.path.join(un_dir, maskname))
            os.rename(m_path.replace('_masks.png', '.jpg'), os.path.join(un_dir, maskname.replace('_masks.png', '.jpg')))

def manage_unused(ui_dir, i_dir, uf_dir, f_dir, min_masks):
    add_unused_img(ui_dir, i_dir, min_masks)
    add_unused_csv(uf_dir, f_dir, min_masks)
    check_unused_img(ui_dir, i_dir, min_masks)
    check_unused_csv(uf_dir, f_dir, min_masks)
    
for i in c2_list:
    c2 = i
    
    if img_folder == 'H&EStain':
        model_name = "heDRG123_"+str(min_masks)+"masks_"+c1[0]+c2[0]+'2_'+initial_model
    else:
        model_name = "transDRG123_"+str(min_masks)+"masks_"+c1[0]+c2[0]+'2_'+initial_model
    
    train_dir = os.path.join(base_dir, img_folder, 'Train')
    test_dir = os.path.join(base_dir, img_folder, 'Test')
    
    if img_folder == 'H&EStain':
        eval_dir = '/media/yhs/5596744f-db7c-442f-9235-d0c9d50c0a6b/Cellpose/Updated_Figures/Evaluation Image/DRG4HeImg'
        pred_path = eval_dir + '/csv/drg4.csv'
        man_path = '/media/yhs/5596744f-db7c-442f-9235-d0c9d50c0a6b/Xenium_segementation/ManuscriptProject/Scaled_DRG_Manual_Annotations/drg4_manual_scaled.csv'
        out_path_drg4 = '/media/yhs/5596744f-db7c-442f-9235-d0c9d50c0a6b/Cellpose/Updated_Figures/Channel Comparison/he/drg4/'+model_name+'_'+str(flow_threshold)+'_'+str(cellprob_threshold)+'drg4.csv'
        out_path_test = '/media/yhs/5596744f-db7c-442f-9235-d0c9d50c0a6b/Cellpose/Updated_Figures/Channel Comparison/he/test/'+model_name+'_'+str(flow_threshold)+'_'+str(cellprob_threshold)+'test.csv'
        heat_map_out_path = '/media/yhs/5596744f-db7c-442f-9235-d0c9d50c0a6b/Cellpose/Updated_Figures/Default Prob Maps/he_trained_'+model_name+'_'+str(flow_threshold)+'_'+str(cellprob_threshold)+'drg4.tif'
    else:
        eval_dir = '/media/yhs/5596744f-db7c-442f-9235-d0c9d50c0a6b/Cellpose/Updated_Figures/Evaluation Image/TransDRG4'
        pred_path = eval_dir + '/csv/croppedDRG4.csv'
        man_path = '/media/yhs/5596744f-db7c-442f-9235-d0c9d50c0a6b/Cellpose/Updated_Figures/Evaluation Image/ManualDRG4/drg4manual.csv'
        out_path_drg4 = '/media/yhs/5596744f-db7c-442f-9235-d0c9d50c0a6b/Cellpose/Updated_Figures/Channel Comparison/trans/drg4/'+model_name+'_'+str(flow_threshold)+'_'+str(cellprob_threshold)+'drg4.csv'
        out_path_test = '/media/yhs/5596744f-db7c-442f-9235-d0c9d50c0a6b/Cellpose/Updated_Figures/Channel Comparison/trans/test/'+model_name+'_'+str(flow_threshold)+'_'+str(cellprob_threshold)+'test.csv'
        heat_map_out_path = '/media/yhs/5596744f-db7c-442f-9235-d0c9d50c0a6b/Cellpose/Updated_Figures/Default Prob Maps/trans_trained_'+model_name+'_'+str(flow_threshold)+'_'+str(cellprob_threshold)+'drg4.tif'
    
    #RUN THIS CELL TO INITIALIZE ALL REQUIRED FILES
    
    directory = os.path.join(base_dir, txt_folder)
    folders = [os.path.join(directory, entry) for entry in os.listdir(directory) if os.path.isdir(os.path.join(directory, entry))]
    
    for folder in folders:
        f_dir = os.pa th.join(directory, folder)
        i_dir = f_dir.replace(txt_folder, img_folder)
        ui_dir = os.path.join(i_dir, 'unused')
        uf_dir = os.path.join(f_dir, 'unused')
        os.makedirs(ui_dir, exist_ok=True)
        os.makedirs(uf_dir, exist_ok=True)
        for filename, imgname in list_mapping([i for i in os.listdir(f_dir) if '.txt' in i], [j for j in os.listdir(i_dir) if '.jpg' in j]):
            file_path = os.path.join(f_dir, filename)
            img_path = os.path.join(i_dir, imgname)
            data = extracting_rois(file_path, img_path)
            df = {'Name': [], 'X': [], 'Y': []}
            count = 1
            for i in data:
                df['Name'].extend(['roi'+str(count) for j in i])
                df['X'].extend([k for j, k in i])
                df['Y'].extend([j for j, k in i])
                count += 1
            df_csv = pd.DataFrame(df)
            df_csv.to_csv(os.path.join(f_dir, filename.replace('.txt','.csv')))
        for filename, imgname in list_mapping([i for i in os.listdir(f_dir) if '.csv' in i], [j for j in os.listdir(i_dir) if '.jpg' in j]):
            file_path = os.path.join(f_dir, filename)
            img_path = os.path.join(i_dir, imgname)
            imarr = csv_png(file_path, 500, 500)
            im = Image.fromarray(imarr)
            im.save(os.path.join(img_path.replace('.jpg','_masks.png')))
    
    #FILTERS NEW CELLS
    for folder in folders:
        f_dir = os.path.join(directory, folder)
        i_dir = f_dir.replace(txt_folder, img_folder)
        ui_dir = os.path.join(i_dir, 'unused')
        uf_dir = os.path.join(f_dir, 'unused')
        manage_unused(ui_dir, i_dir, uf_dir, f_dir, min_masks)
    
    # other parameters for training.
    #@markdown ###Training Parameters:
    #@markdown Number of epochs:
    n_epochs =  100#@param {type:"number"}
    
    Channel_to_use_for_training = c1 #@param ["Grayscale", "Blue", "Green", "Red"]
    Second_training_channel= c2 #@param ["None", "Blue", "Green", "Red"]
    
    #@markdown ###Advanced Parameters
    
    Use_Default_Advanced_Parameters = True #@param {type:"boolean"}
    #@markdown ###If not, please input:
    learning_rate = 0.1 #@param {type:"number"}
    weight_decay = 0.0001 #@param {type:"number"}
    
    if (Use_Default_Advanced_Parameters):
      print("Default advanced parameters enabled")
      learning_rate = 0.1
      weight_decay = 0.0001
    
    #here we check that no model with the same name already exist, if so delete
    model_path = train_dir + '/models'
    if os.path.exists(model_path+'/'+model_name):
      print("!! WARNING: "+model_name+" already exists and will be deleted in the following cell !!")
    
    if len(test_dir) == 0:
      test_dir = None
    
    # Here we match the channel to number
    if Channel_to_use_for_training == "Grayscale":
      chan = 0
    elif Channel_to_use_for_training == "Blue":
      chan = 3
    elif Channel_to_use_for_training == "Green":
      chan = 2
    elif Channel_to_use_for_training == "Red":
      chan = 1
    
    
    if Second_training_channel == "Blue":
      chan2 = 3
    elif Second_training_channel == "Green":
      chan2 = 2
    elif Second_training_channel == "Red":
      chan2 = 1
    elif Second_training_channel == "None":
      chan2 = 0
    
    if initial_model=='scratch':
      initial_model = 'None'
        
    
    # start logger (to see training across epochs)
    logger = io.logger_setup()
    
    # DEFINE CELLPOSE MODEL (without size model)
    model = models.CellposeModel(gpu=use_GPU, model_type=initial_model)
    
    # set channels
    channels = [chan, chan2]
    
    # get files
    output = io.load_train_test_data(train_dir, test_dir, mask_filter='_masks')
    train_data, train_labels, _, test_data, test_labels, _ = output
    
    nimg = len(train_data)
    
    new_model_path = train.train_seg(model.net, train_data=train_data,
                                  train_labels=train_labels,
                                  test_data=test_data,
                                  test_labels=test_labels,
                                  channels=channels,
                                  save_path=train_dir,
                                  n_epochs=n_epochs,
                                  learning_rate=learning_rate,
                                  weight_decay=weight_decay,
                                  SGD=True,                            
                                  nimg_per_epoch=8,
                                  min_train_masks=min_masks,
                                  model_name=model_name)
    
    # diameter of labels in training images
    diam_labels = model.net.diam_labels.item()
    
    # get files (during training, test_data is transformed so we will load it again)
    output = io.load_train_test_data(test_dir, mask_filter='_masks')
    test_data, test_labels = output[:2]
        
    # run model on test images
    masks = model.eval(test_data,
                       channels=[chan, chan2],
                       diameter=diam_labels)[0]
    
    # check performance using ground truth labels
    ap = metrics.average_precision(test_labels, masks)[0]

    print(f'>>> average precision at iou threshold 0.5 = {ap[:,0].mean():.3f}')
    
    print(diam_labels)
    
    # model name and path
    
    #@markdown ###Custom model path (full path):
    
    model_path = os.path.join(train_dir, 'models', model_name)
    
    #@markdown ###Path to images:
    
    dir = eval_dir #@param {type:"string"}
    
    #@markdown ###Channel Parameters:
    
    Channel_to_use_for_segmentation = c1 #@param ["Grayscale", "Blue", "Green", "Red"]
    
    # @markdown If you have a secondary channel that can be used, for instance nuclei, choose it here:
    
    Second_segmentation_channel= c2 #@param ["None", "Blue", "Green", "Red"]
    
    
    # Here we match the channel to number
    if Channel_to_use_for_segmentation == "Grayscale":
      chan = 0
    elif Channel_to_use_for_segmentation == "Blue":
      chan = 3
    elif Channel_to_use_for_segmentation == "Green":
      chan = 2
    elif Channel_to_use_for_segmentation == "Red":
      chan = 1
    
    
    if Second_segmentation_channel == "Blue":
      chan2 = 3
    elif Second_segmentation_channel == "Green":
      chan2 = 2
    elif Second_segmentation_channel == "Red":
      chan2 = 1
    elif Second_segmentation_channel == "None":
      chan2 = 0
    
    #@markdown ### Segmentation parameters:
    
    #@markdown diameter of cells (set to zero to use diameter from training set):
    diameter =  diam_labels #@param {type:"number"}
    
    
    # gets image files in dir (ignoring image files ending in _masks)
    files = io.get_image_files(dir, '_masks')
    images = [io.imread(f) for f in files]
    
    # declare model
    model = models.CellposeModel(gpu=True,
                                 pretrained_model=model_path)
    
    # use model diameter if user diameter is 0
    diameter = model.diam_labels if diameter==0 else diameter
    
    # run model on test images
    masks, flows, styles = model.eval(images,
                                      channels=[chan, chan2],
                                      diameter=diameter,
                                      flow_threshold=flow_threshold,
                                      cellprob_threshold=cellprob_threshold
                                      )
    
    io.masks_flows_to_seg(images,
                          masks,
                          flows,
                          files,
                          channels=[chan, chan2],
                          diams=diameter*np.ones(len(masks)),
                          )
    
    save_rois_with_conversion(masks, files, eval_dir)
    
    zip_to_csv(eval_dir)
    
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f'DRG4 Eval Complete in {elapsed_time:.2f} seconds')
    
    start_time = time.time()
    if img_folder == 'H&EStain':
        performance = optimized_model_eval_files(man_path, pred_path)
    else:
        performance = optimized_model_eval_files(man_path, pred_path)
    
    df_csv = pd.DataFrame.from_dict(performance)
    df_csv.to_csv(out_path_drg4)
    end_time = time.time()
    
    elapsed_time = end_time-start_time
    
    plt.figure(figsize=(8, 10))
    plt.imshow(flows[0][2], cmap='viridis', aspect='auto')
    plt.colorbar(label='Intensity')
    plt.title('Heatmap Visualization')
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.show()
    plt.savefig(heat_map_out_path)
    
    print(f'Eval Pipeline Complete in {elapsed_time:.2f} seconds')
    
    # model name and path
    #duplicate the test folder and label it manual
    
    man_dir_1 = os.path.join(base_dir, txt_folder, 'Test')
    eval_dir_1 = test_dir.replace('Test', 'Eval')
    if os.path.isdir(eval_dir_1):
        shutil.rmtree(eval_dir_1)
    shutil.copytree(test_dir, eval_dir_1)
    
    print(txt_folder)
    
    eu_dir = os.path.join(eval_dir_1, 'unused')
    tt_dir = os.path.join(base_dir, txt_folder, 'Test')
    utt_dir = os.path.join(tt_dir, 'unused')
    manage_unused(eu_dir, eval_dir_1, utt_dir, tt_dir, 0)
    
    #@markdown ###Custom model path (full path):
    
    model_path = os.path.join(train_dir, 'models', model_name)
    
    #@markdown ###Path to images:
    
    dir = eval_dir_1 #@param {type:"string"}
    
    #@markdown ###Channel Parameters:
    
    Channel_to_use_for_segmentation = c1 #@param ["Grayscale", "Blue", "Green", "Red"]
    
    # @markdown If you have a secondary channel that can be used, for instance nuclei, choose it here:
    
    Second_segmentation_channel= c2 #@param ["None", "Blue", "Green", "Red"]
    
    
    # Here we match the channel to number
    if Channel_to_use_for_segmentation == "Grayscale":
      chan = 0
    elif Channel_to_use_for_segmentation == "Blue":
      chan = 3
    elif Channel_to_use_for_segmentation == "Green":
      chan = 2
    elif Channel_to_use_for_segmentation == "Red":
      chan = 1
    
    
    if Second_segmentation_channel == "Blue":
      chan2 = 3
    elif Second_segmentation_channel == "Green":
      chan2 = 2
    elif Second_segmentation_channel == "Red":
      chan2 = 1
    elif Second_segmentation_channel == "None":
      chan2 = 0
    
    #@markdown ### Segmentation parameters:
    
    #@markdown diameter of cells (set to zero to use diameter from training set):
    diameter =  diam_labels #@param {type:"number"}
    
    
    # gets image files in dir (ignoring image files ending in _masks)
    files = io.get_image_files(dir, '_masks')
    images = [io.imread(f) for f in files]
    
    # declare model
    model = models.CellposeModel(gpu=True,
                                 pretrained_model=model_path)
    
    # use model diameter if user diameter is 0
    diameter = model.diam_labels if diameter==0 else diameter
    
    # run model on test images
    masks, flows, styles = model.eval(images,
                                      channels=[chan, chan2],
                                      diameter=diameter,
                                      flow_threshold=flow_threshold,
                                      cellprob_threshold=cellprob_threshold
                                      )
    
    io.masks_flows_to_seg(images,
                          masks,
                          flows,
                          files,
                          channels=[chan, chan2],
                          diams=diameter*np.ones(len(masks)),
                          )
    
    save_rois_with_conversion(masks, files, eval_dir_1)
    
    zip_to_csv(eval_dir_1)
    
    if not os.path.isdir(os.path.join(man_dir_1, 'csv')):
        zip_to_csv(man_dir_1)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f'Test Folder Eval Complete in {elapsed_time:.2f} seconds')
    
    start_time = time.time()
    
    performance_1 = optimized_model_eval(man_dir_1, os.path.join(eval_dir_1, 'csv'))
    
    df_csv = pd.DataFrame.from_dict(performance_1)
    df_csv.to_csv(out_path_test)
    end_time = time.time()
    
    elapsed_time = end_time-start_time
    
    print(f'Eval Pipeline Complete in {elapsed_time:.2f} seconds')

Ready
2025-05-25 13:39:00,067 [INFO] ** TORCH CUDA version installed and working. **
>>> GPU activated? YES
None
[('DRG_1_SegTrans_3000_1500.txt', 'DRG_1_SegTrans_3000_1500.jpg'), ('DRG_2_SegTrans_0_4000.txt', 'DRG_2_SegTrans_0_4000.jpg'), ('DRG_1_SegTrans_1500_1500.txt', 'DRG_1_SegTrans_1500_1500.jpg'), ('DRG_3_SegTrans_1000_1500.txt', 'DRG_3_SegTrans_1000_1500.jpg'), ('DRG_2_SegTrans_0_3000.txt', 'DRG_2_SegTrans_0_3000.jpg'), ('DRG_1_SegTrans_1500_2500.txt', 'DRG_1_SegTrans_1500_2500.jpg')]
[('DRG_2_SegTrans_0_4000.csv', 'DRG_2_SegTrans_0_4000.jpg'), ('DRG_1_SegTrans_1500_2500.csv', 'DRG_1_SegTrans_1500_2500.jpg'), ('DRG_1_SegTrans_1500_1500.csv', 'DRG_1_SegTrans_1500_1500.jpg'), ('DRG_3_SegTrans_1000_1500.csv', 'DRG_3_SegTrans_1000_1500.jpg'), ('DRG_1_SegTrans_3000_1500.csv', 'DRG_1_SegTrans_3000_1500.jpg'), ('DRG_2_SegTrans_0_3000.csv', 'DRG_2_SegTrans_0_3000.jpg')]
[('DRG_3_SegTrans_3000_1500.txt', 'DRG_3_SegTrans_3000_1500.jpg'), ('DRG_1_SegTrans_2000_2500.txt', 'DRG_1_SegTrans_2

100%|███████████████████████████████████████████| 49/49 [00:02<00:00, 22.76it/s]

2025-05-25 13:39:04,412 [INFO] computing flows for labels



100%|█████████████████████████████████████████████| 8/8 [00:00<00:00, 23.51it/s]

2025-05-25 13:39:04,759 [INFO] >>> computing diameters



100%|███████████████████████████████████████████| 8/8 [00:00<00:00, 1104.49it/s]

2025-05-25 13:39:04,813 [INFO] >>> using channels [1, 0]
2025-05-25 13:39:04,814 [INFO] >>> normalizing {'lowhigh': None, 'percentile': None, 'normalize': True, 'norm3D': True, 'sharpen_radius': 0, 'smooth_radius': 0, 'tile_norm_blocksize': 0, 'tile_norm_smooth3D': 1, 'invert': False}


2025-05-25 13:39:05,251 [INFO] >>> n_epochs=100, n_train=49, n_test=8
2025-05-25 13:39:05,253 [INFO] >>> SGD, learning_rate=0.10000, weight_decay=0.00010, momentum=0.900
2025-05-25 13:39:05,258 [INFO] >>> saving model to /media/yhs/5596744f-db7c-442f-9235-d0c9d50c0a6b/Cellpose/Batch2/DRG123/UnremovedTranscripts/Train/models/transDRG123_5masks_RN2_cyto2


OutOfMemoryError: CUDA out of memory. Tried to allocate 50.00 MiB. GPU 

In [6]:
import torch
torch.cuda.empty_cache()